In [1]:
import pathlib
from concurrent.futures import ProcessPoolExecutor, as_completed

import pandas as pd

from bolero_process.atac.sc.fragments_to_zarr import *

In [2]:
dataset = "Domcke2020Science"
genome = "hg38"
cpu = 36
chrom_sizes_path = f"/ref/{genome}/fasta/{genome}.main.chrom.sizes"

In [3]:
frag_paths = pd.Series(
    {".".join(p.name.split(".")[:-3]): p for p in pathlib.Path(f"/tempdata/{dataset}/").glob("*.fragments.txt.gz")}
)
frag_paths.size

72

In [ ]:
for chunk_start in range(0, frag_paths.size, cpu):
    with ProcessPoolExecutor(cpu) as exe:
        chunk_paths = frag_paths.iloc[chunk_start : chunk_start + cpu]
        fs = []
        for sample, path in chunk_paths.items():
            f = exe.submit(
                fragments_to_cutsite_zarr,
                fragments_path=path,
                chrom_sizes_path=chrom_sizes_path,
                output_zarr_path=f"/wmb/zarr/{dataset}.CutSites/{sample}.zarr",
                barcode_prefix=sample,
                format="bed",
                adjust_tn5=False,
                chunk_size=20000000,
            )
            fs.append(f)
        for f in as_completed(fs):
            f.result()